In [1]:
import yfinance as yf
import pyspark as ps
import pandas as pd
from pyspark.sql import SparkSession

Téléchargement de la data

In [2]:
aapl_data = yf.Ticker("AAPL")
historical_data = aapl_data.history(period = "1mo")

historical_data.to_csv("apple.csv")

In [3]:
spark = SparkSession.builder \
    .appName("CSV Operations") \
    .getOrCreate()

24/10/02 14:17:03 WARN Utils: Your hostname, MacBook-Air-de-Virginia.local resolves to a loopback address: 127.0.0.1; using 10.5.27.56 instead (on interface en0)
24/10/02 14:17:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/02 14:17:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Création Data Frame Spark 

In [4]:
df = spark.read.csv("apple.csv", header=True)
df.printSchema()

24/10/02 14:17:21 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Dividends: string (nullable = true)
 |-- Stock Splits: string (nullable = true)

